In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from collections import Counter
import nbformat

2024-12-03 23:03:50.857749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733267031.080385   37780 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733267031.145962   37780 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 23:03:51.800054: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%run '/home/ec2-user/kaggle/rudolph/dataPrep.ipynb'

Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a 

In [3]:
print(stories[0][0])
print(stories[0][1])

A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.
"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."
Do not attempt too much at once.



Return to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle
Or read more short stories for kids in ourChildren's Library
into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . more half put he ourChildren's filberts satisfied much . D

In [3]:
# Sample dataset: Pairs of original and scrambled sentences 
"""
sentences = [
    ("I love programming", "programming I love"),
    ("Keras makes deep learning easy", "makes easy Keras deep learning"),
    ("The cat sat on the mat", "on the mat The cat sat"),
    ("Hello world", "world Hello"),
    ("Pine Cone and Pepper Pot ran after her. Jeremy Mouse only stuck his head out of the snow cave. He didn’t want Jack Frost to nip his toes and certainly not his tail! Jack Frost! Jack Frost! cried Tiptoes. We’ve come to pay a visit. Jack Frost landed next to them. He was pale, icy blue, and his arms and fingers were long and glistening. His eyes were sharp, and his wings crackled like breaking ice whenever he moved. He looked so fierce that Pine Cone and Pepper Pot were afraid to open their mouths. It became so chilly they wrapped their beards round their necks and pulled their hats over their ears.", "snow he beards only fingers so out their We’ve of and came his sharp, they Cone her. afraid glistening. he Jack long, their It Jack He open his Pot! was to Pot and hats nip after not He necks the ears. Jeremy over Jack next icy Mouse round moved. He tail! was pale, the pay their icy pine wrapped their didn’t heads ice fierce glistening. mouths. breaking over pay Jack next like arms his moved. and certainly! only out was the look Tiptoes over not mouths. and he his to Pepper afraid cave. him Jeremy stuck crackled came blue, pulled ran his round with tail! He them. Mouse after didn’t nip fingers wings their not tail! mouse Pot afraid stuck arms of toes icy Cone was them pay. Pine wings came Pepper their He fierce long his wings Cone hats icy his over and next certainly Jack after ears. glistening. pay tiptoes! like pine ran was icy cave! certainly moved the."),
    ("Machine learning is fascinating", "fascinating Machine is learning"),
]
"""
sentences = stories

In [ ]:
#print(sentences[0])
#print(stories[0])


('I love programming', 'programming I love')
('A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.\n"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."\nDo not attempt too much at once.\n\n\n\nReturn to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle\nOr read more short stories for kids in ourChildren\'s Library', 'into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . 

In [4]:
#Original from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add, Dense

# Add <start> and <end> tokens to the original sentences
original_sentences = [f"<start> {s[0]} <end>" for s in sentences]
scrambled_sentences = [s[1] for s in sentences]

# Tokenize and preprocess sentences
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(original_sentences + scrambled_sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

max_seq_length = max(
    max(len(seq.split()) for seq in scrambled_sentences),
    max(len(seq.split()) for seq in original_sentences),
)

# Convert sentences to sequences and pad
input_sequences = pad_sequences(tokenizer.texts_to_sequences(scrambled_sentences), maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(tokenizer.texts_to_sequences(original_sentences), maxlen=max_seq_length, padding="post")

# Create target sequences for teacher forcing
target_input_sequences = target_sequences[:, :-1]
target_output_sequences = target_sequences[:, 1:]

# Define model parameters
embedding_dim = 200
latent_dim = 200
num_heads = 4

# Define Encoder
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs = Dense(latent_dim)(encoder_embedding)  # Encoder output for attention

# Define Decoder with Transformer architecture
decoder_inputs = Input(shape=(max_seq_length - 1,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)

# Self-Attention in Decoder
decoder_self_attention = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_embedding, decoder_embedding)
decoder_self_attention = Add()([decoder_embedding, decoder_self_attention])  # Residual connection
decoder_self_attention = LayerNormalization()(decoder_self_attention)

# Cross-Attention between Encoder and Decoder
decoder_cross_attention = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_self_attention, encoder_outputs)
decoder_cross_attention = Add()([decoder_self_attention, decoder_cross_attention])  # Residual connection
decoder_cross_attention = LayerNormalization()(decoder_cross_attention)

# Feed-forward layer
decoder_ffn = Dense(latent_dim * 4, activation="relu")(decoder_cross_attention)
decoder_ffn = Dense(latent_dim)(decoder_ffn)
decoder_ffn = Add()([decoder_cross_attention, decoder_ffn])  # Residual connection
decoder_ffn = LayerNormalization()(decoder_ffn)

# Final output layer
decoder_outputs = Dense(vocab_size, activation="softmax")(decoder_ffn)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Prepare data for training
decoder_target_output = np.expand_dims(target_output_sequences, -1)
X_train, X_test, y_train, y_test, decoder_y_train, decoder_y_test = train_test_split(
    input_sequences, target_input_sequences, decoder_target_output, test_size=0.2, random_state=42
)






2024-12-03 23:04:58.810025: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
# New Add <start> and <end> tokens to the original sentences
original_sentences = [f"<start> {s[0]} <end>" for s in sentences]
scrambled_sentences = [s[1] for s in sentences]

# Tokenize and preprocess sentences
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(original_sentences + scrambled_sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

max_seq_length = max(
    max(len(seq.split()) for seq in scrambled_sentences),
    max(len(seq.split()) for seq in original_sentences),
)

# Convert sentences to sequences and pad
input_sequences = pad_sequences(tokenizer.texts_to_sequences(scrambled_sentences), maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(tokenizer.texts_to_sequences(original_sentences), maxlen=max_seq_length, padding="post")

# Create target sequences for teacher forcing
target_input_sequences = target_sequences[:, :-1]
target_output_sequences = target_sequences[:, 1:]

embedding_dim = 256
latent_dim = 256
num_heads = 4

# Encoder
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs = Dense(latent_dim)(encoder_embedding)

# Additional encoder layer
encoder_dense_2 = Dense(latent_dim, activation="relu")(encoder_outputs)
encoder_outputs = Add()([encoder_outputs, encoder_dense_2])  # Residual connection
encoder_outputs = LayerNormalization()(encoder_outputs)

# Decoder
decoder_inputs = Input(shape=(max_seq_length - 1,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)

# Self-Attention
decoder_self_attention = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim)(decoder_embedding, decoder_embedding)
decoder_self_attention = Add()([decoder_embedding, decoder_self_attention])
decoder_self_attention = LayerNormalization()(decoder_self_attention)

# Cross-Attention
decoder_cross_attention = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim)(decoder_self_attention, encoder_outputs)
decoder_cross_attention = Add()([decoder_self_attention, decoder_cross_attention])
decoder_cross_attention = LayerNormalization()(decoder_cross_attention)

# Feed-forward layers
decoder_ffn = Dense(latent_dim * 4, activation="relu")(decoder_cross_attention)
decoder_ffn = Dense(latent_dim)(decoder_ffn)
decoder_ffn = Add()([decoder_cross_attention, decoder_ffn])  # Residual connection
decoder_ffn = LayerNormalization()(decoder_ffn)

# Additional feed-forward block
decoder_ffn_2 = Dense(latent_dim * 4, activation="relu")(decoder_ffn)
decoder_ffn_2 = Dense(latent_dim)(decoder_ffn_2)
decoder_ffn_2 = Add()([decoder_ffn, decoder_ffn_2])  # Residual connection
decoder_ffn_2 = LayerNormalization()(decoder_ffn_2)

# Output layer
decoder_outputs = Dense(vocab_size, activation="softmax")(decoder_ffn_2)

# Compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Prepare data for training
decoder_target_output = np.expand_dims(target_output_sequences, -1)
X_train, X_test, y_train, y_test, decoder_y_train, decoder_y_test = train_test_split(
    input_sequences, target_input_sequences, decoder_target_output, test_size=0.2, random_state=42
)


In [7]:
# Train the model
print("Training the model...")
model.fit(
    [X_train, y_train], decoder_y_train,
    validation_data=([X_test, y_test], decoder_y_test),
    batch_size=60,
    epochs=50,
)

Training the model...
Epoch 1/50


1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5353 - loss: 3.2404 - val_accuracy: 0.4976 - val_loss: 3.8861
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5368 - loss: 3.1796 - val_accuracy: 0.4999 - val_loss: 3.8801
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5521 - loss: 3.1179 - val_accuracy: 0.4992 - val_loss: 3.8727
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.5582 - loss: 3.0593 - val_accuracy: 0.5001 - val_loss: 3.8715
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5716 - loss: 3.0009 - val_accuracy: 0.5021 - val_loss: 3.8731
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5804 - loss: 2.9439 - val_accuracy: 0.5020 - val_loss: 3.8735
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5845 - loss: 2.8869 - val_accuracy: 0.5016 - val_loss: 3.8779
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5892 - loss: 2.8338 - val_accuracy: 0.5017 - val_loss: 3.8822
Epoch 9/50


In [12]:
# Save the model in SavedModel format
model.save("/home/ec2-user/kaggle/rudolph/model_v0.keras")

In [13]:



# Define inference models
# Encoder model remains the same
encoder_model = Model(encoder_inputs, encoder_outputs)

# Decoder for inference
decoder_inputs_inf = Input(shape=(max_seq_length - 1,))
decoder_embedding_inf = Embedding(vocab_size, embedding_dim)(decoder_inputs_inf)

decoder_self_attention_inf = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_embedding_inf, decoder_embedding_inf)
decoder_self_attention_inf = Add()([decoder_embedding_inf, decoder_self_attention_inf])
decoder_self_attention_inf = LayerNormalization()(decoder_self_attention_inf)

decoder_cross_attention_inf = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_self_attention_inf, encoder_outputs)
decoder_cross_attention_inf = Add()([decoder_self_attention_inf, decoder_cross_attention_inf])
decoder_cross_attention_inf = LayerNormalization()(decoder_cross_attention_inf)

decoder_ffn_inf = Dense(latent_dim * 4, activation="relu")(decoder_cross_attention_inf)
decoder_ffn_inf = Dense(latent_dim)(decoder_ffn_inf)
decoder_ffn_inf = Add()([decoder_cross_attention_inf, decoder_ffn_inf])
decoder_ffn_inf = LayerNormalization()(decoder_ffn_inf)

decoder_outputs_inf = Dense(vocab_size, activation="softmax")(decoder_ffn_inf)

decoder_model = Model([decoder_inputs_inf, encoder_outputs], decoder_outputs_inf)

In [14]:
# Define function to decode scrambled sentences
def decode_sequence(input_seq, scrambled_sentence):
    """
    Decode a scrambled sentence while ensuring the output is a permutation
    of the words in the scrambled sentence.
    """
    # Encode the input to get the context vector
    encoder_output = encoder_model.predict(input_seq)

    # Tokenize the scrambled sentence to get word counts
    scrambled_words = scrambled_sentence.split()
    word_count = Counter(scrambled_words)

    # Generate the initial target sequence with the start token
    target_seq = np.zeros((1, max_seq_length - 1))
    target_seq[0, 0] = tokenizer.word_index["<start>"]

    # Initialize variables for decoding
    stop_condition = False
    decoded_sentence = []
    used_words = set()

    while not stop_condition:
        # Predict the next token probabilities
        output_tokens = decoder_model.predict([target_seq, encoder_output])

        # Create a mask for valid words
        valid_word_indices = []
        for word, count in word_count.items():
            if count > 0:  # Only include words still available in scrambled input
                token_index = tokenizer.word_index.get(word, None)
                if token_index is not None:
                    valid_word_indices.append(token_index)

        # Mask out probabilities of invalid words
        if valid_word_indices:
            mask = np.ones(output_tokens[0, -1, :].shape, dtype=bool)
            mask[valid_word_indices] = False  # Only allow valid word indices
            output_tokens[0, -1, mask] = 0
        else:
            # No valid words left, terminate decoding
            stop_condition = True
            continue

        # Resample a valid word
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, None)

        # Ensure the sampled word is valid and not repeating excessively
        if sampled_word in word_count and word_count[sampled_word] > 0:
            decoded_sentence.append(sampled_word)
            word_count[sampled_word] -= 1
            used_words.add(sampled_word)

        # Debugging prints
        print(f"Resampled token index: {sampled_token_index}")
        print(f"Resampled word: {sampled_word}")

        # Check if decoding is complete
        if len(decoded_sentence) == len(scrambled_words) or sum(word_count.values()) == 0:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, max_seq_length - 1))
        target_seq[0, 0] = sampled_token_index

    # Validate that all words were used; if not, add missing words
    missing_words = [word for word in scrambled_words if word not in used_words]
    decoded_sentence.extend(missing_words)

    return " ".join(decoded_sentence).strip()




In [18]:
# Test the model
#scrambled_input = "row your boat down the stream gently row row"
scrambled_input ="on the sat mat cat the"
#scrambled_input ="are ? you where"

input_sequence = pad_sequences(tokenizer.texts_to_sequences([scrambled_input]), maxlen=max_seq_length, padding="post")
unscrambled_sentence = decode_sequence(input_sequence, scrambled_input)
print(f"Scrambled: {scrambled_input}")
print(f"Unscrambled: {unscrambled_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Resampled token index: 151
Resampled word: sat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
Resampled token index: 434
Resampled word: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
Resampled token index: 17
Resampled word: on
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
Scrambled: on the sat mat cat the
Unscrambled: sat cat on the the mat


In [ ]:
"""
scrambled_sentence = "on the mat The cat sat"
scrambled_words = scrambled_sentence.split()
print(Counter(scrambled_words))
print(original_sentences)
print(scrambled_sentences)
"""

Counter({'on': 1, 'the': 1, 'mat': 1, 'The': 1, 'cat': 1, 'sat': 1})
['<start> I love programming <end>', '<start> Keras makes deep learning easy <end>', '<start> The cat sat on the mat <end>', '<start> Hello world <end>', '<start> Machine learning is fascinating <end>']
['programming I love', 'makes easy Keras deep learning', 'on the mat The cat sat', 'world Hello', 'fascinating Machine is learning']
